# SQL project
<p>In this project, I will take a look at the `chinook.db` database and will formulate a few business questions, which I will subsequently answer.</p>

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

First, I will have a look at the names of the tables within the database and at a few of the tables listed.

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


In [3]:
%%sql
SELECT *
FROM customer
LIMIT 5;

Done.


customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5
3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
4,Bjørn,Hansen,None,Ullevålsveien 14,Oslo,None,Norway,0171,+47 22 44 22 22,None,bjorn.hansen@yahoo.no,4
5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4


In [4]:
%%sql
SELECT *
FROM invoice_line
LIMIT 5;

Done.


invoice_line_id,invoice_id,track_id,unit_price,quantity
1,1,1158,0.99,1
2,1,1159,0.99,1
3,1,1160,0.99,1
4,1,1161,0.99,1
5,1,1162,0.99,1


### First business question:
Find out which genres sell the most tracks in the USA. Of the fours genres "Hip-Hop", "Punk", "Pop", and "Blues", which is the least popular genre?

In [5]:
%%sql
SELECT 
        t.genre_id,
        COUNT(i.quantity) sold_count
FROM invoice_line i
INNER JOIN track t ON t.track_id = i.track_id
GROUP BY 1
LIMIT 5;

Done.


genre_id,sold_count
1,2635
2,121
3,619
4,492
6,124


In [6]:
%%sql
SELECT il.invoice_id,
        il.track_id,
        il.quantity
FROM invoice_line il
INNER JOIN invoice i ON il.invoice_id = i.invoice_id
INNER JOIN customer c ON c.customer_id = i.customer_id
WHERE c.country = 'USA'
LIMIT 5;

Done.


invoice_id,track_id,quantity
1,1158,1
1,1159,1
1,1160,1
1,1161,1
1,1162,1


In [7]:
%%sql
WITH track_counts AS (
SELECT 
        t.genre_id,
        COUNT(il.invoice_line_id) sold_count
FROM invoice_line il
INNER JOIN track t ON t.track_id = il.track_id
INNER JOIN invoice i ON il.invoice_id = i.invoice_id
INNER JOIN customer c ON c.customer_id = i.customer_id
WHERE c.country = 'USA'
GROUP BY 1
)    


SELECT 
        g.name genre,
        tc.sold_count
FROM genre g
INNER JOIN track_counts tc ON tc.genre_id = g.genre_id
ORDER BY 2 DESC
LIMIT 10;

Done.


genre,sold_count
Rock,561
Alternative & Punk,130
Metal,124
R&B/Soul,53
Blues,36
Alternative,35
Latin,22
Pop,22
Hip Hop/Rap,20
Jazz,14


Based on this tables, the genre "Hip Hop" is the least favourite one.

## Second business question:
Which sales agent performed best? 

To find out which sales agent performed best, I will need to combine the tables `employee`, `customer`, and `invoice`:

In [8]:
%%sql
WITH total_per_customer AS (
SELECT 
        customer_id,
        SUM(total) AS total_spent
    FROM invoice
    GROUP BY customer_id
),

customer_support AS (
SELECT c.customer_id,
        c.support_rep_id,
        SUM(t.total_spent) total_spent_per_support
    FROM customer c
INNER JOIN total_per_customer t ON t.customer_id = c.customer_id
GROUP BY support_rep_id
)

SELECT e.first_name || " " || e.last_name AS employee,
        cs.total_spent_per_support AS total_per_employee
FROM employee e
INNER JOIN customer_support cs ON e.employee_id = cs.support_rep_id
LIMIT 5;


Done.


employee,total_per_employee
Jane Peacock,1731.5099999999998
Margaret Park,1584.0000000000002
Steve Johnson,1393.92


There are only three employees. Of these three, Jane sold the most records followed by Margeret and Steve.

## Third business question:
For each country, what is the
<ul>
<li> total number of customers</li>
<li> total value of sales</li>
<li> average value of sales per customer</li>
<li> average order value</li>
</ul>
The countries with only one customer will be grouped to "Other".

In [9]:
%%sql
WITH country_and_other AS (
SELECT
    CASE
    WHEN (
        SELECT COUNT(customer_id) AS count
        FROM customer
        WHERE country = c.country
    ) = 1 THEN "Other"
    ELSE c.country
    END AS country_case,
    c.customer_id,
    i.invoice_id,
    il.unit_price AS price   
FROM customer c
INNER JOIN invoice i ON i.customer_id = c.customer_id
INNER JOIN invoice_line il ON il.invoice_id = i.invoice_id
)
SELECT  country_case,
        COUNT(customer_id) AS customer,
        SUM(price) sales,
        SUM(price)/COUNT(DISTINCT invoice_id) AS avg_order_value,
        SUM(price)/COUNT(DISTINCT customer_id) AS avg_sales_per_customer
FROM country_and_other
GROUP BY country_case
ORDER BY customer
LIMIT 4;
        

Done.


country_case,customer,sales,avg_order_value,avg_sales_per_customer
India,185,183.1500000000002,8.72142857142858,91.5750000000001
Portugal,187,185.13000000000022,6.3837931034482835,92.56500000000011
United Kingdom,248,245.52000000000078,8.768571428571457,81.84000000000026
Czech Republic,276,273.24000000000103,9.108000000000034,136.62000000000052


## Fourth business question
What percentage of purchases are individual tracks vs whole albums?

Is it better to offer whole albumns or to offer the most popular tracks? In this analysis I will discard albumns with only a few tracks, as the business question focuses on whether it is worth having all titles of an albumn in store. 

First, I will need to identify whether each invoice contains all the tracks of said albumn to find out whether the whole albumn has been purchased or only a few tracks. This is best done using `EXCEPT` and `AND`. This business question is more complicated than the previous ones, therefore I will explain every step:

First, I need some kind of album track identifyier, which I can use to filter for the albumn and see whether all tracks of the invoice are on the albumn. Looking at `invoice_line`, I choose the following:

In [10]:
%%sql
SELECT il.invoice_id,
        MIN(il.track_id) track_identifyer
FROM invoice_line il
GROUP BY il.invoice_id
LIMIT 5

Done.


invoice_id,track_identifyer
1,1158
2,201
3,2516
4,748
5,1986


Next step is to look whether the invoices contain all albumn tracks: 
<ul>
<li> album of track(s) purchased</li>
<li> track(s) purchased </li>
<li> are these the same?</li>
<ul>

In [27]:
%%sql
WITH invoice_track_identifyer AS (
SELECT invoice_id,
        track_id,
        MIN(track_id) track_identifyer
FROM invoice_line 
GROUP BY invoice_id
)
SELECT t.track_id
FROM track t
INNER JOIN invoice_track_identifyer iti ON iti.track_id = t.track_id
WHERE t.album_id = (
                    SELECT t2.album_id FROM track t2
                    WHERE t2.track_id = iti.track_identifyer
)
LIMIT 2;

Done.


track_id
1158
201


In [24]:
%%sql
WITH invoice_track_identifyer AS (
SELECT invoice_id,
        track_id,
        MIN(track_id) track_identifyer
FROM invoice_line 
GROUP BY invoice_id
)

SELECT il.track_id
FROM invoice_line il
INNER JOIN invoice_track_identifyer iti ON iti.invoice_id = il.invoice_id
WHERE il.invoice_id = iti.invoice_id
LIMIT 2;

Done.


track_id
1158
1159


In [31]:
%%sql
WITH invoice_track_identifyer AS (
SELECT invoice_id,
        track_id,
        MIN(track_id) track_identifyer
FROM invoice_line 
GROUP BY invoice_id
)

SELECT t.track_id
FROM track t
INNER JOIN invoice_track_identifyer iti ON iti.track_id = t.track_id
WHERE t.album_id = (
                    SELECT t2.album_id FROM track t2
                    WHERE t2.track_id = iti.track_identifyer
)
EXCEPT

SELECT il.track_id
FROM invoice_line il
INNER JOIN invoice_track_identifyer iti ON iti.invoice_id = il.invoice_id
WHERE il.invoice_id = iti.invoice_id

Done.


[]

This part will be part of a CASE clause, which will iterate over the rows. If a customer bought an album, the other way (tracts bought EXCEPT tracks on album) must be equals as well:

In [33]:
%%sql
WITH invoice_track_identifyer AS (
SELECT invoice_id,
        track_id,
        MIN(track_id) track_identifyer
FROM invoice_line 
GROUP BY invoice_id
)


SELECT il.track_id
FROM invoice_line il
INNER JOIN invoice_track_identifyer iti ON iti.invoice_id = il.invoice_id
WHERE il.invoice_id = iti.invoice_id

EXCEPT

SELECT t.track_id
FROM track t
INNER JOIN invoice_track_identifyer iti ON iti.track_id = t.track_id
WHERE t.album_id = (
                    SELECT t2.album_id FROM track t2
                    WHERE t2.track_id = iti.track_identifyer
)
LIMIT 2;

Done.


track_id
25
41


Now I need to combine both cells above in a CASE statement:

In [35]:
%%sql
WITH invoice_track_identifyer AS (
SELECT invoice_id,
        track_id,
        MIN(track_id) track_identifyer
FROM invoice_line 
GROUP BY invoice_id
)

SELECT *,
    CASE
        WHEN
            (
            SELECT t.track_id
            FROM track t
            
            WHERE t.album_id = (
                                SELECT t2.album_id FROM track t2
                                WHERE t2.track_id = iti.track_identifyer
            )
            EXCEPT

            SELECT il.track_id
            FROM invoice_line il
            
            WHERE il.invoice_id = iti.invoice_id
            ) IS NULL    
        AND
                
            (SELECT il.track_id
            FROM invoice_line il
            
            WHERE il.invoice_id = iti.invoice_id

            EXCEPT

            SELECT t.track_id
            FROM track t
            
            WHERE t.album_id = (
                                SELECT t2.album_id FROM track t2
                                WHERE t2.track_id = iti.track_identifyer
            )) IS NULL
        THEN "yes"
        ELSE "no"
    END AS "full_album_bought"
    FROM invoice_track_identifyer iti
    GROUP BY invoice_id
    LIMIT 3;
            
            

Done.


invoice_id,track_id,track_identifyer,full_album_bought
1,1158,1158,yes
2,201,201,no
3,2516,2516,no


The last step is to compute the percentage of invoices with "yes" and "no":

In [42]:
%%sql
WITH invoice_track_identifyer AS (
SELECT invoice_id,
        track_id,
        MIN(track_id) track_identifyer
FROM invoice_line 
GROUP BY invoice_id
)

SELECT
      full_album_bought,
        CAST(COUNT(full_album_bought) AS float)/
                (SELECT COUNT(*) FROM invoice) AS percentage
FROM (
    SELECT iti.invoice_id,
        CASE
            WHEN
                (
                SELECT t.track_id
                FROM track t

                WHERE t.album_id = (
                                    SELECT t2.album_id FROM track t2
                                    WHERE t2.track_id = iti.track_identifyer
                )
                EXCEPT

                SELECT il.track_id
                FROM invoice_line il

                WHERE il.invoice_id = iti.invoice_id
                ) IS NULL    
            AND

                (SELECT il.track_id
                FROM invoice_line il

                WHERE il.invoice_id = iti.invoice_id

                EXCEPT

                SELECT t.track_id
                FROM track t

                WHERE t.album_id = (
                                    SELECT t2.album_id FROM track t2
                                    WHERE t2.track_id = iti.track_identifyer
                )) IS NULL
            THEN "yes"
            ELSE "no"
        END AS "full_album_bought"
        FROM invoice_track_identifyer iti
        GROUP BY invoice_id
        
) 
GROUP BY full_album_bought

Done.


full_album_bought,percentage
no,0.8143322475570033
yes,0.18566775244299674


It seems that most people buy single tracks instead of full albums.